In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."


In [ ]:

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]


In [3]:

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

print(response)

# Save function call outputs for subsequent requests
input_list += response.output

print(input_list)



Response(id='resp_02d3b6d94b16fb3b0068e17d84a2588196b4e7b8d30d3da98e', created_at=1759608196.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_02d3b6d94b16fb3b0068e17d84fe7481968a994f741dc1b2cd', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_dqnokFRRVUTtT9D661RaHlOC', name='get_horoscope', type='function_call', id='fc_02d3b6d94b16fb3b0068e17d864a18819685f360f0ecefdce4', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_horoscope', parameters={'type': 'object', 'properties': {'sign': {'type': 'string', 'description': 'An astrological sign like Taurus or Aquarius'}}, 'required': ['sign'], 'additionalProperties': False}, strict=True, type='function', description="Get today's horoscope for an astrological sign.")], to

In [4]:
for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)


Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_02d3b6d94b16fb3b0068e17d84fe7481968a994f741dc1b2cd', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_dqnokFRRVUTtT9D661RaHlOC', name='get_horoscope', type='function_call', id='fc_02d3b6d94b16fb3b0068e17d864a18819685f360f0ecefdce4', status='completed'), {'type': 'function_call_output', 'call_id': 'call_dqnokFRRVUTtT9D661RaHlOC', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."}'}]


In [5]:

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final output:
{
  "id": "resp_02d3b6d94b16fb3b0068e17da47004819698cb30f7f83b8a0c",
  "created_at": 1759608228.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_02d3b6d94b16fb3b0068e17da4e60c8196a467f17423b01625",
      "content": [
        {
          "annotations": [],
          "text": "{'sign': 'Aquarius'}: Next Tuesday you will befriend a baby otter.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_horoscope",
      "parameters": {
        "type": "object",
        "properties": {
          "sign": {
            "type": "string",
            "description": "An astrologic